# Cek GPU

In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA Available: True
GPU: NVIDIA A100-SXM4-40GB


# Install YOLO V8

In [2]:
!pip install ultralytics
from ultralytics import YOLO
YOLO("yolov8s.pt")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

100%|██████████| 21.5M/21.5M [00:00<00:00, 40.7MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

# Connect Google Drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Extract Dataset

In [4]:
import zipfile
import os

dataset_zip = "/content/drive/MyDrive/Project_Kelompok6/dataset/data.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(dataset_zip, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)

Dataset extracted to: /content/dataset


# Cek struktur dataset

In [5]:
os.listdir(extract_path)

['__MACOSX', 'labels', 'images']

# Membagi Dataset (Train, Val, Test)

In [6]:
import shutil
import os
import glob
from sklearn.model_selection import train_test_split

base_dir = "/content/dataset"
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
    raise FileNotFoundError("⚠️ Folder images atau labels tidak ditemukan. Cek kembali path dataset!")

output_dir = "/content/split_dataset"
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_dir, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, split, "labels"), exist_ok=True)

image_files = sorted(glob.glob(os.path.join(images_dir, "*.*")))
label_files = sorted(glob.glob(os.path.join(labels_dir, "*.txt")))

image_names = set(os.path.splitext(os.path.basename(img))[0] for img in image_files)
label_names = set(os.path.splitext(os.path.basename(lbl))[0] for lbl in label_files if not lbl.startswith("."))  # Hindari file sistem

valid_images = [img for img in image_files if os.path.splitext(os.path.basename(img))[0] in label_names]
valid_labels = [lbl for lbl in label_files if os.path.splitext(os.path.basename(lbl))[0] in image_names]

if len(valid_images) == 0 or len(valid_labels) == 0:
    raise ValueError("⚠️ Dataset kosong atau tidak ditemukan gambar yang sesuai dengan label!")

train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.10

train_imgs, test_imgs = train_test_split(valid_images, test_size=test_ratio, random_state=42)
train_imgs, val_imgs = train_test_split(train_imgs, test_size=val_ratio / (train_ratio + val_ratio), random_state=42)

def copy_files(files, dest_folder):
    for file in files:
        shutil.copy(file, dest_folder)

for split, imgs in zip(["train", "val", "test"], [train_imgs, val_imgs, test_imgs]):
    copy_files(imgs, os.path.join(output_dir, f"{split}/images"))

    labels = [os.path.join(labels_dir, os.path.basename(os.path.splitext(f)[0]) + ".txt") for f in imgs]
    copy_files(labels, os.path.join(output_dir, f"{split}/labels"))

print("\n✅ Dataset berhasil dibagi! 🚀\n")
print(f"🔹 Train: {len(train_imgs)} gambar")
print(f"🔹 Validation: {len(val_imgs)} gambar")
print(f"🔹 Test: {len(test_imgs)} gambar")


✅ Dataset berhasil dibagi! 🚀

🔹 Train: 3750 gambar
🔹 Validation: 750 gambar
🔹 Test: 500 gambar


# Konversi Anotasi dari XML (VOC) ke TXT (YOLO Format)

In [8]:
import xml.etree.ElementTree as ET

def convert_voc_to_yolo(xml_folder, img_folder, label_folder, class_mapping):
    for xml_file in os.listdir(xml_folder):
        if not xml_file.endswith(".xml"):
            continue

        tree = ET.parse(os.path.join(xml_folder, xml_file))
        root = tree.getroot()
        image_name = root.find("filename").text
        image_path = os.path.join(img_folder, image_name)

        img_w = int(root.find("size/width").text)
        img_h = int(root.find("size/height").text)

        label_file = os.path.join(label_folder, xml_file.replace(".xml", ".txt"))
        with open(label_file, "w") as f:
            for obj in root.findall("object"):
                class_name = obj.find("name").text
                if class_name not in class_mapping:
                    continue

                class_id = class_mapping[class_name]
                bndbox = obj.find("bndbox")
                xmin = int(bndbox.find("xmin").text)
                ymin = int(bndbox.find("ymin").text)
                xmax = int(bndbox.find("xmax").text)
                ymax = int(bndbox.find("ymax").text)

                x_center = (xmin + xmax) / 2 / img_w
                y_center = (ymin + ymax) / 2 / img_h
                width = (xmax - xmin) / img_w
                height = (ymax - ymin) / img_h

                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

class_mapping = {"With Helmet": 0, "Without Helmet": 1}

for split in ["train", "val", "test"]:
    convert_voc_to_yolo(
        xml_folder=os.path.join(output_dir, split, "labels"),
        img_folder=os.path.join(output_dir, split, "images"),
        label_folder=os.path.join(output_dir, split, "labels"),
        class_mapping=class_mapping
    )

print("Konversi selesai!")

Konversi selesai!


# Buat File data.yaml untuk YOLO

In [9]:
yaml_content = """train: /content/split_dataset/train/images
val: /content/split_dataset/val/images
test: /content/split_dataset/test/images

nc: 2
names: ["With Helmet", "Without Helmet"]
"""

with open("/content/dataset.yaml", "w") as f:
    f.write(yaml_content)

print("dataset.yaml berhasil dibuat!")

dataset.yaml berhasil dibuat!


# Augmentasi Dataset


In [1]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

model.train(
    data="/content/dataset.yaml",
    epochs=100,
    batch=16,
    imgsz=640,

    weight_decay=0.0005,
    patience=30,

    augment=True,
    degrees=10,
    fliplr=0.5,
    flipud=0.1,
    mosaic=0.5,
    hsv_h=0.015, hsv_s=0.5, hsv_v=0.3,

    optimizer="AdamW",
    lr0=0.005,
    lrf=0.1,

    project="/content/training",
    name="helmet_detection",
    save=True,
    save_period=5,
    val=True
)

Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/dataset.yaml, epochs=100, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=/content/training, name=helmet_detection2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=

train: Scanning /content/split_dataset/train/labels... 3750 images, 313 backgrounds, 0 corrupt: 100%|██████████| 3750/3750 [00:04<00:00, 784.06it/s]


train: New cache created: /content/split_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/split_dataset/val/labels... 750 images, 58 backgrounds, 0 corrupt: 100%|██████████| 750/750 [00:01<00:00, 623.26it/s]

val: New cache created: /content/split_dataset/val/labels.cache


...

                   all        750       2971      0.942      0.908       0.96      0.634



100 epochs completed in 0.735 hours.
Optimizer stripped from /content/training/helmet_detection2/weights/last.pt, 22.5MB
Optimizer stripped from /content/training/helmet_detection2/weights/best.pt, 22.5MB

Validating /content/training/helmet_detection2/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:06<00:00,  3.92it/s]


                   all        750       2971      0.949        0.9      0.961      0.632
           With Helmet        692       2971      0.949        0.9      0.961      0.632
Speed: 0.1ms preprocess, 3.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/training/helmet_detection2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa3b6601710>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# Download Best.pt

In [2]:
from google.colab import files

files.download('/content/training/helmet_detection2/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>